In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import boto3
import json
import matplotlib.pyplot as plt
import requests
import pathlib
import utils.configs as configs
import utils.strava as strava

plt.style.use('seaborn-v0_8-pastel')

In [3]:
s3 = boto3.resource('s3')
bucket_name = "strava-raw"

resp = strava.auth().json()
access_token = resp['access_token']

In [4]:
activity_ids = []
for obj in s3.Bucket(bucket_name).objects.all():
    if pathlib.Path(obj.key).stem.isnumeric():
        body_bytes = obj.get()['Body'].read()
        activity_ids.append(json.loads(body_bytes)['id'])

In [17]:
""" Tables: zones, laps, and streams
    I invision that these will become three tables adjacent to the existing "activities" table
    The three new tables branch from the activities table, where id is primary key for all
"""
tables = ["zones", "laps", "streams"]

for table in tables:
    data = strava.batch_get_request(table, activity_ids, access_token) 
    break
json.dump(data, open("test.json", 'w'))

Executing request 0 of 258
Executing request 10 of 258
Executing request 20 of 258
Executing request 30 of 258
Executing request 40 of 258
Executing request 50 of 258
Executing request 60 of 258
Executing request 70 of 258
Executing request 80 of 258
Executing request 90 of 258
Executing request 100 of 258
Executing request 110 of 258
Executing request 120 of 258
Executing request 130 of 258
Executing request 140 of 258
Executing request 150 of 258
Executing request 160 of 258
Executing request 170 of 258
Executing request 180 of 258
Executing request 190 of 258
Executing request 200 of 258
Executing request 210 of 258
Executing request 220 of 258
Executing request 230 of 258
Executing request 240 of 258
Executing request 250 of 258
